In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# Prepare Data set for Cats and Dogs:

'1. Data preprocessing: Image Generator, Train Test Split'
'2. Model architecting'
'3. Model compile'
'4. Model Traiing'
'5. Model Testing / Input testing'

'5. Model Testing / Input testing'

In [4]:
train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

In [8]:
train_image_data = train_image_generator.flow_from_directory('cats_and_dogs/cats_and_dogs/train',
                                                             class_mode= 'binary',
                                                             target_size = (64,64),
                                                             batch_size = 10
                                                             )

test_image_data = test_image_generator.flow_from_directory('cats_and_dogs/cats_and_dogs/validation', class_mode='binary',
                                                           target_size = (64,64),
                                                           batch_size = 10)

Found 2000 images belonging to 2 classes.
Found 1002 images belonging to 2 classes.


In [9]:

vgg = tf.keras.applications.vgg16.VGG16(include_top=False)
vgg.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# We want to retain the existing weights of the existing model | We need not to train them:
# This can be archived using: layer.trainble = False

for layer in vgg.layers:
    layer.trainable = False

In [ ]:
# #Create our custom model: In that create Input layer
input_layer = tf.keras.layers.Input(shape=[255,255,3]) # Decleration of Input layer

x = vgg(input_layer) # Now X contains vegg with Input layer

x1= tf.keras.layers.Flatten()(x) # Flatten the Convolution layer ip -> vgg -> flatten

x2 = tf.keras.layers.Dense(activation= 'relu', units = 128, name= 'h1')(x1) # First ANN layer 

x3 = tf.keras.layers.Dense(activation = 'relu', units = 4096, name = 'h2')(x2)

output = tf.keras.layers.Dense(activation='sigmoid', units = 1, name = 'output')(x3)

# Combine all (input and output) these layers and then create a model:

model = tf.keras.models.Model(inputs = input_layer, outputs = output)

model.summary()



ValueError: Unrecognized keyword arguments passed to Model: {'input': <KerasTensor shape=(None, 255, 255, 3), dtype=float32, sparse=False, ragged=False, name=keras_tensor_61>, 'output': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, ragged=False, name=keras_tensor_83>}